In [1]:
import pandas as pd

In [2]:
uniprot_mapping = pd.read_csv('raw_data/uniprot-yourlist_M201910086746803381A1F0E0DB47453E0216320D0044DCI+active_yes.tab', sep='\t')
hippie = pd.read_csv('raw_data/hippie_current.txt', sep='\t', names=['A', 'Aa', 'B', 'Bb', 'score', 'evidence'])
location_data = pd.read_excel('raw_data/41467_2015_BFncomms8866_MOESM610_ESM.xlsx', 1)

In [ ]:
display(uniprot_mapping[:3])
display(location_data[:3])
display(hippie[:3])

In [ ]:
merged = pd.merge(uniprot_mapping, location_data, how='left', left_on='Entry', right_on='UniProtAC')
merged = merged[['Entry', 'Entry name', 'Protein names', 'Gene names', 'Consensus_SL']]
merged['Consensus_SL'] = merged['Consensus_SL'].str.split('\.\s')

In [ ]:
merged[merged['Entry'] == "O15455"]

In [ ]:
hippie_merged = pd.merge(hippie, uniprot_mapping[['Entry name', 'Entry']], how='left', left_on='A', right_on='Entry name')
hippie_merged = pd.merge(hippie_merged, uniprot_mapping[['Entry name', 'Entry']], how='left', left_on='B', right_on='Entry name', suffixes=('_A','_B'))
hippie_merged = hippie_merged.dropna(subset=['Entry_A', 'Entry_B'])[['Entry_A', 'Entry_B', 'score']]

In [ ]:
def find_interactions(mapping_row):
    ac = mapping_row['Entry']
    right = hippie_merged[hippie_merged['Entry_A'] == ac].values
    left = hippie_merged[hippie_merged['Entry_B'] == ac].values
    
    return [{'partner': x[1], 'score': x[2]} for x in right] + [{'partner': x[0], 'score': x[2]} for x in left]

In [ ]:
merged['partners'] = merged.apply(find_interactions, axis=1)

In [ ]:
merged.to_csv('data.csv', index=False)

In [ ]:
merged.to_json('data.json', orient='records')